#Görev Tanımı
Bu görevde, verilen ahşap yüzey verileri kullanılarak denetimsiz öğrenme (unsupervised learning) yaklaşımıyla bir anomali tespit modeli geliştirilecektir. Model, ahşap yüzeyinde kusurların veya düzensizliklerin tespit edilmesi için eğitilecektir.


#1. Ön İşleme Adımları ve Veri Hazırlığı


In [16]:
import cv2
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import os

# Google Drive'ı bağla
drive.mount('/content/drive')

# Drive'dan veri setine erişim
data_path = '/content/drive/MyDrive/wood_dataset/'

# train/good klasörüne erişim
train_good_path = os.path.join(data_path, 'wood/train/good/')

# test/good klasörüne erişim
test_good_path = os.path.join(data_path, 'wood/test/good/')

# test/defect klasörüne erişim
test_defect_path = os.path.join(data_path, 'wood/test/defect/')

# ground_truth/defect klasörüne erişim
ground_truth_defect_path = os.path.join(data_path, 'wood/ground_truth/defect/')

# İşlenmiş resimler için çıkış klasörü
output_dir = 'processed_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Arka Planı Kaldırma


In [ ]:
def crop_and_clean(image):
    # Gri tonlamaya çevir
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Otsu Threshold uygula
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Siyah olmayan alanları bul
    coords = cv2.findNonZero(thresh)

    if coords is not None:
        # En küçük dikdörtgeni belirle ve kırp
        x, y, w, h = cv2.boundingRect(coords)
        cropped_image = image[y:y+h, x:x+w]

        # İkinci kontrol
        gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        _, clean_thresh = cv2.threshold(gray_cropped, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Yeniden kırp
        new_coords = cv2.findNonZero(clean_thresh)
        if new_coords is not None:
            x2, y2, w2, h2 = cv2.boundingRect(new_coords)
            cropped_image = cropped_image[y2:y2+h2, x2:x2+w2]

    return cropped_image


###Görüntü Normalizasyonu ve Boyutlandırma

In [ ]:
def resize_and_normalize(image, target_size=(256, 256)):
    # Boyutlandırma
    resized_image = cv2.resize(image, target_size)

    # Normalizasyon [0, 1] aralığına
    normalized_image = resized_image / 255.0

    return normalized_image


In [ ]:
def process_image(image):
    # Kırpma ve temizleme
    cropped_image = crop_and_clean(image)

    # Boyutlandırma ve normalizasyon
    final_image = resize_and_normalize(cropped_image)


    return final_image


In [17]:
# 'train/good' klasöründeki tüm resimleri işleyip kaydetme
for filename in os.listdir(train_good_path):
    if filename.endswith(('jpg', 'png')):  # Sadece resim dosyaları
        img_path = os.path.join(train_good_path, filename)

        # Resmi oku
        image = cv2.imread(img_path)

        # Resmi işleme (crop ve normalize)
        processed_image = process_image(image)

        # İşlenmiş resmi kaydet
        output_filename = os.path.join(output_dir, f"processed_{filename}")
        processed_image = (processed_image * 255).astype(np.uint8)  # [0, 1] -> [0, 255]
        cv2.imwrite(output_filename, cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR))  # OpenCV BGR formatında

        print(f"{output_filename} başarıyla kaydedildi.")

processed_images/processed_11.jpg başarıyla kaydedildi.
processed_images/processed_6.jpg başarıyla kaydedildi.
processed_images/processed_4.jpg başarıyla kaydedildi.
processed_images/processed_5.jpg başarıyla kaydedildi.
processed_images/processed_9.jpg başarıyla kaydedildi.
processed_images/processed_10.jpg başarıyla kaydedildi.
processed_images/processed_7.jpg başarıyla kaydedildi.
processed_images/processed_8.jpg başarıyla kaydedildi.
processed_images/processed_12.jpg başarıyla kaydedildi.
processed_images/processed_19.jpg başarıyla kaydedildi.
processed_images/processed_24.jpg başarıyla kaydedildi.
processed_images/processed_16.jpg başarıyla kaydedildi.
processed_images/processed_20.jpg başarıyla kaydedildi.
processed_images/processed_30.jpg başarıyla kaydedildi.
processed_images/processed_29.jpg başarıyla kaydedildi.
processed_images/processed_22.jpg başarıyla kaydedildi.
processed_images/processed_23.jpg başarıyla kaydedildi.
processed_images/processed_28.jpg başarıyla kaydedildi

In [18]:
import os
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('jpg', 'png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_files[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR -> RGB

        # İşleme adımları
        image = process_image(image)  # Önceden tanımlanan işlem fonksiyonlarıyla

        if self.transform:
            image = self.transform(image)

        return image

# Resimleri işlemek için dönüşümler
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
])

# Dataset ve DataLoader
dataset_dir = "processed_images"
dataset = CustomImageDataset(image_dir=dataset_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


#2. Denetimsiz Model Eğitimi

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import cv2
import os

In [23]:
!git clone https://github.com/cqylunlun/GLASS.git


Cloning into 'GLASS'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 200 (delta 70), reused 61 (delta 59), pack-reused 118 (from 1)
Receiving objects: 100% (200/200), 8.23 MiB | 17.92 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [25]:
import sys
sys.path.append('/content/GLASS')


In [30]:
from glass import GLASS  # Reposu içerisindeki kodu import et
import torch
import torchvision.models as models

# Cihazı (GPU/CPU) seçin
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ResNet50 modelini yükle
backbone_model = models.resnet50(pretrained=True)  # Önceden eğitilmiş ResNet50
backbone_model = backbone_model.to(device)  # Modeli doğru cihaza (GPU/CPU) yükle

# Modeli başlatma
input_shape = (3, 224, 224)  # Giriş boyutu (3 renk kanalı, 224x224)
target_embed_dimension = 128  # Hedef embed boyutu
pretrain_embed_dimension = 512  # Önceden eğitimli embed boyutu

# GLASS modelini başlat
glass_model = GLASS(device)

# GLASS modeline backbone olarak ResNet50 modelini yükle
glass_model.load(
    backbone=backbone_model,
    layers_to_extract_from=[0, 1, 2],  # Hangi katmanlardan özellik çıkarılacağı
    device=device,
    input_shape=input_shape,
    pretrain_embed_dimension=pretrain_embed_dimension,
    target_embed_dimension=target_embed_dimension
)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 86.6MB/s]


TypeError: argument of type 'int' is not iterable